# Food Service quality in the Dutch suburbs and business opportunities there

Coursera Capstone Project, part of the Data Sciences path of IBM

In [1]:
#run needed installs
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.1-py36_0          conda-forge
    ca-certificates: 2019.10.16-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.9.11-py36_0  --> 2019.11.28-py36_0     conda-f

In [2]:
#import needed libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim


import folium
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.preprocessing import StandardScaler, normalize, scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

print('Libraries imported!')

Libraries imported!


I. Project Summary:
Using data science techniques to analyze the following questions:

- What is the variety and quality of food in Utrecht? (FourSquare API)
- What is the variety and quality of food in Nieuwegein?  (FourSquare API)
- Comparassion of real estates prices Utrecht and Nieuwegein, based on specific drivers (e.g.: Size, Kitchen, Garden) (NVM Data)
- What service could Nieuwegein need to attract more foodies?


II. Preparing data

1) Using FourSquare API to get Utrecht venues:
2) Using FourSquare API to get Nieuwegein venues:
3)

In [3]:
#Using FourSquare API to get  venues

# define the dataframe columns
column_names = ['City', 'Latitude', 'Longitude'] 

# instantiate the dataframe
cities = pd.DataFrame(columns=column_names)

In [9]:
cities

,City,Latitude,Longitude
0,Nieuwegein,52.029870,5.092933
1,Utrecht,52.080986,5.127684


In [5]:
addressN = 'Nieuwegein, Utrecht, Nederland, Nederland'
geolocator = Nominatim(user_agent="foursquare_agent")
locationN = geolocator.geocode(addressN)
latitudeN = locationN.latitude
longitudeN = locationN.longitude
print("Nieuwegein: ", latitudeN, longitudeN)


addressU = 'Utrecht, Nederland, Nederland'
locationU = geolocator.geocode(addressU)
latitudeU = locationU.latitude
longitudeU = locationU.longitude
print("Utrecht: ", latitudeU, longitudeU)

Nieuwegein:  52.0298704 5.09293340872922
Utrecht:  52.0809856 5.12768396945229


In [7]:
#cities = cities.append({'City' : 'Nieuwegein' , 'Latitude' : 52.0298704 , 'Longitude':5.09293340872922} , ignore_index=True)
#cities = cities.append({'City' : 'Utrecht' , 'Latitude' : 52.0809856 , 'Longitude': 5.12768396945229} , ignore_index=True)
cities

,City,Latitude,Longitude
0,Nieuwegein,52.029870,5.092933
1,Utrecht,52.080986,5.127684


In [11]:
# create map of New York using latitude and longitude values
mapNU = folium.Map(location=[latitudeU, longitudeU], zoom_start=10)

# add markers to map
for lat, lng, city in zip(cities['Latitude'], cities['Longitude'], cities['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapNU)
    
mapNU

In [12]:
CLIENT_ID = '0N455GT5MN0KZIZGGCIQBJEL2AF0HMZIJVD0KIPM0IZYQEGR' # your Foursquare ID
CLIENT_SECRET = '15AALY2XPKPUYXXPBUBVJ5YV3UOV1GK2VAAJNAR5M2M5UTM1' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0N455GT5MN0KZIZGGCIQBJEL2AF0HMZIJVD0KIPM0IZYQEGR
CLIENT_SECRET:15AALY2XPKPUYXXPBUBVJ5YV3UOV1GK2VAAJNAR5M2M5UTM1


In [13]:
#define limit and radius
LIMIT = 100
radius = 500 

#Utrecht venues
urlU = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitudeU, 
    longitudeU, 
    radius, 
    LIMIT)
urlU # display URL

#Nieuwegein venues
urlN = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitudeN, 
    longitudeN, 
    radius, 
    LIMIT)
urlN # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=0N455GT5MN0KZIZGGCIQBJEL2AF0HMZIJVD0KIPM0IZYQEGR&client_secret=15AALY2XPKPUYXXPBUBVJ5YV3UOV1GK2VAAJNAR5M2M5UTM1&v=20180604&ll=52.0298704,5.09293340872922&radius=500&limit=100'

In [14]:
resultsU = requests.get(urlU).json()
resultsU

{'meta': {'code': 200, 'requestId': '5dea206a1e152c001b57fcce'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Oost',
  'headerFullLocation': 'Oost, Utrecht',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 52.0854856045, 'lng': 5.134992771256711},
   'sw': {'lat': 52.076485595499996, 'lng': 5.120375167647869}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aa8100af964a520104f20e3',
       'name': 'Louis Hartlooper Complex',
       'location': {'address': 'Tolsteegbrug 1',
        'lat': 52.08175209677094,
        'lng': 5.124147029629959,
        'labeledLatLngs': [{'label': 'display',
          'lat': 52.08175209677094

In [15]:
resultsN = requests.get(urlN).json()
resultsN

{'meta': {'code': 200, 'requestId': '5dea2079c8cff245ee0937bf'},
 'response': {'headerLocation': 'Nieuwegein',
  'headerFullLocation': 'Nieuwegein',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 52.0343704045, 'lng': 5.100233852932517},
   'sw': {'lat': 52.0253703955, 'lng': 5.085632964525923}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be52aaf2457a5939b36ab15',
       'name': 'Nettorama',
       'location': {'address': 'Hildo Kropstraat 1-3',
        'lat': 52.03197810789086,
        'lng': 5.089348813291146,
        'labeledLatLngs': [{'label': 'display',
          'lat': 52.03197810789086,
          'lng': 5.089348813291146}],
        'distance': 339,
        'postalCode': '3431 CC',
        'cc': 'NL',
        

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
#define vanues for Utrecht and Nieuwegein
venuesU = resultsU['response']['groups'][0]['items']
venuesN = resultsN['response']['groups'][0]['items']

# flatten JSON
nearby_venuesU = json_normalize(venuesU) 
nearby_venuesN = json_normalize(venuesN)

# filter columns
filtered_columns = ['venue.location.city','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venuesU =nearby_venuesU.loc[:, filtered_columns]
nearby_venuesN =nearby_venuesN.loc[:, filtered_columns]

# filter the category for each row
nearby_venuesU['venue.categories'] = nearby_venuesU.apply(get_category_type, axis=1)
nearby_venuesN['venue.categories'] = nearby_venuesN.apply(get_category_type, axis=1)

# clean columns
nearby_venuesU.columns = [col.split(".")[-1] for col in nearby_venuesU.columns]
nearby_venuesN.columns = [col.split(".")[-1] for col in nearby_venuesN.columns]



In [26]:
nearby_venuesU

,city,name,categories,lat,lng
0,Utrecht,Louis Hartlooper Complex,Indie Movie Theater,52.081752,5.124147
1,Utrecht,Meneer Smakers,Burger Joint,52.081804,5.123941
2,Utrecht,Ledig Erf,Bar,52.081708,5.123739
3,Utrecht,Orloff aan de kade,Café,52.079644,5.123253
4,Utrecht,Centraal Museum,Art Museum,52.083784,5.125964
5,Utrecht,Koffie Leute Brauhaus,Coffee Shop,52.079445,5.122588
6,Utrecht,nijntje museum (Nijntje Museum),Art Museum,52.083961,5.126053
7,Utrecht,Broei,Vegetarian / Vegan Restaurant,52.079285,5.123069
8,Utrecht,Kounosuke,Japanese Restaurant,52.079521,5.122904
9,Utrecht,Fishes,Fish Market,52.083127,5.123518


In [27]:
nearby_venuesN

,city,name,categories,lat,lng
0,Nieuwegein,Nettorama,Supermarket,52.031978,5.089349
1,Nieuwegein,DeMIX Fitness,Gym / Fitness Center,52.026297,5.093290
2,Nieuwegein,Sportcafe Merwestein,Sports Bar,52.026496,5.092953
3,Nieuwegein,Sunbob Nieuwegein,Cosmetics Shop,52.032699,5.089596
4,Nieuwegein,Bushalte Merwesteintunnel,Bus Stop,52.026360,5.095364


In [28]:
print('{} venues were returned by Foursquare in Utrecht.'.format(nearby_venuesU.shape[0]))
print('{} venues were returned by Foursquare in Nieuwegein.'.format(nearby_venuesN.shape[0]))

32 venues were returned by Foursquare in Utrecht.
5 venues were returned by Foursquare in Nieuwegein.


In [29]:
print('There are {} uniques categories in Utrecht.'.format(len(nearby_venuesU['categories'].unique())))
print('There are {} uniques categories in Nieuwegein.'.format(len(nearby_venuesN['categories'].unique())))

There are 24 uniques categories in Utrecht.
There are 5 uniques categories in Nieuwegein.


In [40]:
#combine the dataframes
nearby_venues = pd.concat([nearby_venuesU, nearby_venuesN])
nearby_venues = nearby_venues.reset_index(drop=True)
#nearby_vanues

# one hot encoding
venue_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venue_onehot['city'] = nearby_venues['city'] 

# move neighborhood column to the first column
fixed_columns = [venue_onehot.columns[-1]] + list(venue_onehot.columns[:-1])
venue_onehot = venue_onehot[fixed_columns]

venue_onehot

,city,Art Museum,BBQ Joint,Bakery,Bar,Beer Store,Burger Joint,Bus Stop,Café,Cheese Shop,Coffee Shop,Cosmetics Shop,Falafel Restaurant,Fish Market,French Restaurant,Garden,Gourmet Shop,Grocery Store,Gym / Fitness Center,Ice Cream Shop,Indie Movie Theater,Japanese Restaurant,Science Museum,Snack Place,Sports Bar,Supermarket,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant
0,Utrecht,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Utrecht,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Utrecht,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Utrecht,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Utrecht,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Utrecht,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Utrecht,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Utrecht,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,Utrecht,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,Utrecht,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
venue_grouped = venue_onehot.groupby('city').mean().reset_index()
venue_grouped

,city,Art Museum,BBQ Joint,Bakery,Bar,Beer Store,Burger Joint,Bus Stop,Café,Cheese Shop,Coffee Shop,Cosmetics Shop,Falafel Restaurant,Fish Market,French Restaurant,Garden,Gourmet Shop,Grocery Store,Gym / Fitness Center,Ice Cream Shop,Indie Movie Theater,Japanese Restaurant,Science Museum,Snack Place,Sports Bar,Supermarket,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant
0,Nieuwegein,0.0000,0.00000,0.00000,0.000,0.00000,0.00000,0.2,0.00000,0.00000,0.00000,0.2,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.2,0.00000,0.00000,0.00000,0.00000,0.00000,0.2,0.2000,0.00000,0.00000,0.00000
1,Utrecht,0.0625,0.03125,0.03125,0.125,0.03125,0.03125,0.0,0.09375,0.03125,0.03125,0.0,0.03125,0.03125,0.0625,0.03125,0.03125,0.03125,0.0,0.03125,0.03125,0.03125,0.03125,0.03125,0.0,0.0625,0.03125,0.03125,0.03125


In [43]:
num_top_venues = 5

for hood in venue_grouped['city']:
    print("----"+hood+"----")
    temp = venue_grouped[venue_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Nieuwegein----
                  venue  freq
0           Supermarket   0.2
1            Sports Bar   0.2
2              Bus Stop   0.2
3        Cosmetics Shop   0.2
4  Gym / Fitness Center   0.2


----Utrecht----
               venue  freq
0                Bar  0.12
1               Café  0.09
2         Art Museum  0.06
3        Supermarket  0.06
4  French Restaurant  0.06


